# Use the CUT&RUN models and the ChIP-seq model to predict on the union of all peaks

In [1]:
import os
os.chdir('/home/katie/bp_repo/reports/')

from util import import_shap_scores, import_peak_table, import_profiles, import_tfmodisco_results
from util import trim_motif_by_ic
from util import figure_to_vdom_image
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from IPython.display import display
import tqdm

import sys
sys.path.append('/home/katie/bp_repo/multitask_profile_model_SPI_GATA/')
from plotting_helper import *
from all_functions import *
from profile_models import ModelLoader

tqdm.tqdm_notebook()

plt.rcParams["figure.dpi"] = 130

/tmp/ipykernel_63286/3522789943.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm.tqdm_notebook()


0it [00:00, ?it/s]

## Does the ChIP-seq model predict equally strong peaks on the CUT&RUN-unique peaks vs. ChIP-seq-unique peaks vs. shared peaks?

### Preliminary data generating/loading

In [8]:
# using the CTCF ChIP-seq model!!
# YOU HAVE TO DO DATALOADING SEPARATELY BECAUSE THE TASKS_PATH WILL GET UPDATED (or ig you could update tasks_path in between uses...)

# Save predictions using model and BOTH assays' peaks

# model = ModelLoader('/home/katie/bp_repo/pipeline_outputs/ctcf_chipseq_feb7/' + 'model.state_dict', controls=True, num_tasks=1).load_model()

# chip_dataloader = DataLoader(tasks=['CTCF'], assay='chip-seq', controls=True, 
#                              tasks_path='/home/katie/bp_repo/research/data/chip-seq/', subset=['full'], 
#                              jitter=False, fake_controls=True).make_loaders()['full']

# save_preds(chip_dataloader, model, 
# '/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/chip_model_chip_loader_fake_controls.preds')

# cut_dataloader = DataLoader(tasks=['CTCF'], assay='cutnrun', controls=True, 
#                              tasks_path='/home/katie/bp_repo/research/data/cutnrun/', subset=['full'], 
#                             jitter=False, fake_controls=True).make_loaders()['full']

# save_preds(cut_dataloader, model, 
# '/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/chip_model_cut_loader_fake_controls.preds')


In [2]:
# Generate/load new unique vs shared peaks files
# i don't feel like using the old ones because those contain the chrY and chrM peaks which are hard to deal with

# first, save the peaks files without the chrY and chrM peaks

# # load in peaks
# chip_peaks = import_peak_table(['/home/katie/bp_repo/research/data/chip-seq/CTCF/CTCF.bed'])
# cut_peaks = import_peak_table(['/home/katie/bp_repo/research/data/cutnrun/CTCF/CTCF.bed'])

# # get rid of chrY and chrM peaks
# chip_peaks = chip_peaks.loc[chip_peaks.chrom != 'chrY']
# cut_peaks = cut_peaks.loc[cut_peaks.chrom != 'chrM']

# # clean the dataframe a little bit
# chip_peaks.reset_index(inplace=True, drop=True)
# cut_peaks.reset_index(inplace=True, drop=True)
# chip_peaks.name = chip_peaks.index  # fix weird peak column
# cut_peaks.name = cut_peaks.index # add peak column
# cut_peaks.drop('i9', axis=1, inplace=True)  # remove the i9 column to bring the dataframe down to 20 columns, for import_peak_table()

# chip_peaks.to_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY', sep='\t', index=False)
# cut_peaks.to_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM', sep='\t', index=False)

chip_peaks = pd.read_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY', sep='\t')
cut_peaks = pd.read_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM', sep='\t')

In [3]:
# Generate/load unique vs shared peaks
# !bedtools intersect -v -a /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY -b /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM > /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_no_cut
# !bedtools intersect -v -a /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM -b /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY > /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_no_chip
# !bedtools intersect -u -a /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY -b /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM > /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_shared_cut
# !bedtools intersect -u -a /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_peaks_no_chrM -b /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_peaks_no_chrY > /home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_shared_chip

# btw: unique peaks U shared peaks = all peaks (you can check!)
chip_unique_peaks = import_peak_table(['/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_no_cut'])
cut_unique_peaks = import_peak_table(['/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_no_chip'])
chip_shared_peaks = import_peak_table(['/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_shared_cut'])
cut_shared_peaks = import_peak_table(['/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_shared_chip'])


In [145]:
# Find the duplicate indexes so i can subset all of my preds/counts arrays
chip_coords = pd.DataFrame({'chrom': chip_preds['coords']['coords_chrom'], 
                  'start': chip_preds['coords']['coords_start'],
                  'end': chip_preds['coords']['coords_end']})
chip_coords.chrom = [i.decode('utf-8') for i in chip_coords.chrom]  # change from binary string thingy to utf-8 encoded

cut_coords = pd.DataFrame({'chrom': cut_preds['coords']['coords_chrom'], 
                  'start': cut_preds['coords']['coords_start'],
                  'end': cut_preds['coords']['coords_end']})
cut_coords.chrom = [i.decode('utf-8') for i in cut_coords.chrom]

chip_dups = chip_coords.duplicated()  # boolean array indicating whether a row is a duplicate
cut_dups = cut_coords.duplicated()    

In [154]:
# load in predictions and subset to the unique rows
chip_preds = h5py.File('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/chip_model_chip_loader_fake_controls.preds','r')
cut_preds = h5py.File('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/chip_model_cut_loader_fake_controls.preds','r')

pred_profs_chip = np.exp(np.array(chip_preds['predictions']['log_pred_profs']))[chip_dups]  # each row corresponds to unique peak
pred_profs_cut = np.exp(np.array(cut_preds['predictions']['log_pred_profs']))[cut_dups]
pred_counts_chip = np.exp(np.array(chip_preds['predictions']['log_pred_counts']))[chip_dups]
pred_counts_cut = np.exp(np.array(cut_preds['predictions']['log_pred_counts']))[cut_dups]



## If the predictions look the same, do the importance scores also look the same? (are the same signals being identified as being important?)

## If the predictions are different, what might explain that difference? The TA motif? DNase/ATAC/histone marks?